In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

directory = '/content/drive/MyDrive/text/'
metadata = pd.read_json(directory + 'processed_data/metadata.json')

In [ ]:
len(set([keyword for keywords in metadata['keyword_list'] for keyword in keywords]))

34134

In [ ]:
[keywords for keywords in metadata['keyword_list'] if '세계여행' in keywords]

# Co-occurence based Keyword Clustering
- 있는 정보만 활용
- 문제
  - 1) 새로운 데이터가 들어왔을 때 LV cluster 결과가 달라질 수 있음
  - 2) 새롭게 학습을 진행해야 함

In [ ]:
from collections import Counter

Counter([keyword for keywords in metadata['keyword_list'] for keyword in keywords]).most_common()

[('여행', 33887),
 ('영화', 27058),
 ('사랑', 25177),
 ('에세이', 17629),
 ('그림일기', 15806),
 ('생각', 14254),
 ('공감에세이', 13109),
 ('일상', 11175),
 ('글쓰기', 10086),
 ('감성에세이', 10021),
 ('인생', 8929),
 ('행복', 8472),
 ('연애', 7819),
 ('소설', 7734),
 ('드로잉', 7721),
 ('독서', 7451),
 ('육아', 7253),
 ('그림', 7041),
 ('마음', 7030),
 ('사진', 6686),
 ('리뷰', 6432),
 ('엄마', 6404),
 ('음악', 6089),
 ('스타트업', 5777),
 ('시', 5701),
 ('유럽여행', 5558),
 ('일러스트', 5327),
 ('이별', 5241),
 ('마케팅', 5145),
 ('고양이', 5064),
 ('디자인', 4731),
 ('영화리뷰', 4687),
 ('웹툰', 4646),
 ('IT', 4447),
 ('직장인', 4397),
 ('감성', 4390),
 ('일기', 4330),
 ('사람', 4300),
 ('결혼', 4000),
 ('친구', 3959),
 ('창업', 3917),
 ('회사', 3873),
 ('취업', 3783),
 ('교육', 3719),
 ('감성사진', 3676),
 ('퇴사', 3628),
 ('일본', 3627),
 ('건강', 3500),
 ('세계여행', 3444),
 ('해외여행', 3371),
 ('가족', 3361),
 ('커피', 3355),
 ('다이어트', 3322),
 ('성공', 3280),
 ('시간', 3158),
 ('직장', 3116),
 ('음식', 3081),
 ('운동', 2943),
 ('카페', 2812),
 ('책', 2812),
 ('맛집', 2787),
 ('문학', 2784),
 ('관계', 2773),
 ('가을', 2766),
 

In [ ]:
from itertools import combinations

counted = Counter([tuple(sorted(k)) for pair in metadata['keyword_list'].apply(lambda x: list(combinations(x, 2))) for k in pair])

In [ ]:
import pandas as pd

counted = pd.DataFrame.from_dict(counted.items())
counted[['n1', 'n2']] = pd.DataFrame(counted.iloc[:,0].tolist(), index=counted.index)
counted.rename(columns={1: 'weight'}, inplace=True)

In [ ]:
counted

,0,weight,n1,n2
0,"(여행, 호주)",390,여행,호주
1,"(국립공원, 여행)",36,국립공원,여행
2,"(국립공원, 호주)",2,국립공원,호주
3,"(목련꽃, 아지랑이)",1,목련꽃,아지랑이
4,"(동행, 목련꽃)",1,동행,목련꽃
...,...,...,...,...
705972,"(책임감, 화분)",1,책임감,화분
705973,"(3D, UI)",1,3D,UI
705974,"(3D, 제스처)",1,3D,제스처
705975,"(경험수집, 독서모임)",1,경험수집,독서모임


In [ ]:
import networkx as nx

G = nx.from_pandas_edgelist(counted, 'n1', 'n2', 'weight')

In [ ]:
print(nx.info(G))

Graph with 85894 nodes and 705977 edges


In [ ]:
import matplotlib.pyplot as plt
from community import community_louvain

'''
https://dc.uwm.edu/cgi/viewcontent.cgi?article=1130&context=ijger
에 따르면 키워드 네트워크에 대해 LV이 가장 성능이 좋으며 시간 복잡도도 낮은 편
'''

partition = community_louvain.best_partition(G)

In [ ]:
# set(partition.values()) # 599
# Counter(partition.values())

[k for k, v in partition.items() if v == 2]

['베네수엘라',
 '경제',
 '유치권',
 '주거문화',
 '부동산',
 '적자',
 '강제집행',
 '부동산경매',
 '개봉동',
 '경매',
 '플래카드',
 '사드',
 '평화',
 '김제동',
 '헌법',
 '분쟁',
 '노무현',
 '대통령',
 '노무현입니다',
 '박근혜',
 '블랙리스트',
 '유신헌법',
 '정치',
 '광화문',
 '촛불',
 '유가',
 '주식투자',
 '박원순',
 '서울시장',
 '조류충돌',
 '조류충돌예방',
 '제도',
 '사전투표',
 '투표용지',
 '수출',
 '살림살이',
 '경제학',
 '명도',
 '예산',
 '아파트',
 '동소문동',
 '시연',
 '자본주의',
 '세월호',
 '투자',
 '투자원칙',
 '주식',
 '신용카드소득공제',
 '절세',
 '세금',
 '판사',
 '권력관계',
 '권력',
 '강남',
 '원격의료',
 '노래모임',
 '촛불집회',
 '성주촛불',
 '하멜른',
 '피리',
 '부자',
 '재테크',
 '서울시',
 '소비',
 '절약',
 '추모',
 '법관',
 '선거',
 '법원경매',
 '입찰보증금',
 '민주주의',
 '대박',
 '가계부',
 '신용카드',
 '임팩트',
 '출마',
 '패거리문화',
 '통장',
 '총학생회',
 '공청회',
 '대선',
 '공약집',
 '이사',
 '정주영',
 '국회의원',
 '안철수',
 '미분양',
 '재산세',
 '경제불황',
 '김영란',
 '청탁',
 '도널드트럼프',
 '버락오바마',
 '썰전',
 '저축',
 '서울도서관',
 '서울시청',
 '지자체',
 '금리',
 '단기채권',
 '시중은행',
 '북한',
 '박정희',
 '민주공화국',
 '부동산뉴스',
 '무료취업지원',
 '생애설계',
 '사옥',
 '임대수익률',
 '재건축',
 '도시재생프로젝트',
 '도시재생',
 '국회',
 '대출신청',
 '전세자금',
 '김정은',
 '트럼프',
 '정상회담',
 '금융상품',
 '장기금융시장',
 '파

In [ ]:
metadata['keyword_cluster'] = metadata['keyword_list'].apply(lambda x: set([partition[keyword] if keyword in partition.keys() else -1 for keyword in x]))

In [ ]:
Counter(metadata['keyword_cluster'].apply(len))

Counter({1: 277163, 2: 236056, 0: 70065, 3: 59759, 4: 61})

# SPARQL ontology
- 외부 정보 활용
- 문제
  - 동음이의어 문제

## Ontology Query

In [ ]:
!pip install sparqlwrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 7.7 MB/s 
     |████████████████████████████████| 41 kB 582 kB/s 


In [ ]:
def sparql_query(query_search):

    # query = """
    #     PREFIX dbo:  <http://dbpedia.org/ontology/>
    #     PREFIX dbpedia: <http://ko.dbpedia.org/resource/>
    #     PREFIX dbp: <http://ko.dbpedia.org/property/>
    #     PREFIX dcterms: <http://purl.org/dc/terms/>
    #     PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    #     SELECT distinct (?subject||?redirects) ?comment ?category
    #       WHERE {
    #           ?subject rdfs:comment ?comment.
    #           ?subject dcterms:subject ?category.
    #           ?redirects dbo:wikiPageRedirects* ?subject.
    #           FILTER (?subject = dbpedia:%s || ?redirects = dbpedia:%s)
    #     }
    #     """ % (query_search, query_search)

    query = """
        PREFIX dbo:  <http://dbpedia.org/ontology/>
        PREFIX dbpedia: <http://ko.dbpedia.org/resource/>
        PREFIX dbp: <http://ko.dbpedia.org/property/>
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT * WHERE
        {
          {
            SELECT distinct ?subject ?comment ?category WHERE {
              ?subject rdfs:comment ?comment.
              ?subject dcterms:subject ?category.
              FILTER (?subject = dbpedia:%s)
            }
          }
          UNION
          {
            SELECT distinct ?redirects ?comment ?category WHERE {
              ?subject rdfs:comment ?comment.
              ?subject dcterms:subject ?category.
              ?redirects dbo:wikiPageRedirects* ?subject.
              FILTER (?redirects = dbpedia:%s)
              }
            }
        }
        """ % (query_search, query_search)

    sparql = SPARQLWrapper("http://ko.dbpedia.org/sparql")
    sparql.setQuery(query)

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    return results

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
from tqdm import tqdm
import pickle

# queries_tosearch = [item[0] for item in Counter([keyword for keywords in metadata['keyword_list'] for keyword in keywords]).most_common(50)]
queries_tosearch = set([keyword for keywords in metadata['keyword_list'] for keyword in keywords])
print('Search for...', queries_tosearch)

searched_results = pd.DataFrame()
searched_dict = {}
error_list = {'no_results': [], 'error': []}

for i, query_search in tqdm(enumerate(queries_tosearch)):
    try:
        if query_search not in (set(error_list['no_results']) | set(error_list['error'])):
            results = sparql_query(query_search)
            results_df = pd.json_normalize(results['results']['bindings'])
            # if any(results_df.columns.str.contains('subject.value')):
            #     print(str(i)+'.',query_search, '---->', results_df.shape[0], '건 (Subject)')
            # elif any(results_df.columns.str.contains('redirects.value')):
            #     print(str(i)+'.',query_search, '---->', results_df.shape[0], '건 (Redirects)')
            searched_results = searched_results.append(results_df.loc[:, results_df.columns.str.contains("value")])
            if results_df.shape[0]>0:
                searched_dict[query_search] = set(results_df['category.value'].apply(lambda x: x.replace('http://ko.dbpedia.org/resource/분류:', '')))
            else:
                # new_query_search = spacing(query_search).replace(' ', '_')  
                new_query_search = query_search
                if new_query_search == query_search:
                    # print('-----------------> No results for:', query_search)
                    error_list['no_results'].append(query_search)
                else:
                    # print('-----------------> No results for:', query_search, "=>", new_query_search)
                    error_list['no_results'].append(query_search)
        if i%100 == 0:
            with open(directory + 'error_list.pickle', 'wb') as handle:
                pickle.dump(error_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(directory + 'searched_dict.pickle', 'wb') as handle:
                pickle.dump(searched_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


    except Exception as ex:
        # new_query_search = spacing(query_search).replace(' ', '_')
        print('-----------------> Error:', query_search)
        error_list['error'].append(query_search)

if any(searched_results.columns.str.contains('redirects.value')):
    searched_results = searched_results.drop_duplicates(['redirects.value', 'category.value']).reset_index(drop=True)
if any(searched_results.columns.str.contains('subject.value')):
    searched_results = searched_results.drop_duplicates(['subject.value', 'category.value']).reset_index(drop=True)

Search for... {'물가수준', '리베이트', '편강', '스토브', '임야구입', '울랄라세션', '예정화', '반성글', '조엘오스틴', '도시경제학', '고객서비스', '입시미술', '장르소설', '경주양동마을', '우다이푸르', '바이오의약품', '챗봇서비스', '토론준비', '윤리에세이', '금융공황', '명랑이발관', '씀바귀꽃', '논리철학', '싱가포르뮤직페스티벌', '집안사정', '아침가리', '공의반란', '기사문', '멜로딕데스메탈', '중이염증상', '문화유산보전', '자아도취', '길가', '민족개벽', '셔터스톡', '복소수', '검색포털', '도날드', '대만쇼핑', '시간차', '인문기행', '모둠활동', '노잼', '초능력', '농촌행복복합마을', '생리통', '한국가구', '일치', '뷰티웹툰', '테니스', '전공교육과정', '도그빌', '경력변화전문가', '수사능력', '나무토막', '파리스토리', '기대감', '내수활성화', '아마존ceo', '로빈슨', '은하공책폭탄', '독일새댁', '로렉스', '화랑도서관', '아트콜라보', '타겟팅', '유통혁명', '갑신정변', '음주운전교통사고', '소방박물관', '예비돌싱녀', '풍선마술사', '체인본사', '교통위반', '셀폰', '청춘여행', '이치우미', '개인연금', '선리기연', '대화주제', '파밍로드', '과일', '윤석민', '야채볶음', '시월노트', '스티브', '스테판커리', '맛집정보', '레몬하트', '토종효모빵', '로미오줄리엣', '육아퇴직', '절', '초혼', '피레네산맥', '지진대비', '카디건', '공항픽업', '경희사이버대학교', '빙빙', '부친', '증후군', '일반화', '웨딩카드', '문화한국', '추적자', '헤이즐넛', '독서자본', '법인회생절차', '평화통일교육', '디모데', '기부금영수증', '편집증', '글제목', '권리해태', '배당금지급채권', '부동산거래위임장', '트랜스지방', '주요업무계획', '어혈',

383it [01:10,  6.03it/s]

-----------------> Error: 주식매각


386it [01:10,  6.04it/s]

-----------------> Error: 아해


...

85796it [5:03:28,  4.64it/s]

-----------------> Error: 크루그먼


85799it [5:03:29,  4.44it/s]

-----------------> Error: 인천여중생


86037it [5:04:28,  4.71it/s]


In [ ]:
with open(directory + 'error_list_final.pickle', 'wb') as handle:
    pickle.dump(error_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(directory + 'searched_dict_final.pickle', 'wb') as handle:
    pickle.dump(searched_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
searched_results['subject'] = searched_results[['subject.value', 'redirects.value']].fillna('').sum(axis=1).apply(lambda x: x.replace('http://ko.dbpedia.org/resource/', ''))
searched_results = searched_results[['subject', 'comment.value', 'category.value']].drop_duplicates()
searched_results['category.value'] = searched_results['category.value'].apply(lambda x: x.replace('http://ko.dbpedia.org/resource/분류:', ''))
searched_results = searched_results.groupby(['subject', 'comment.value']).agg({'category.value':lambda x: list(x)}).reset_index()

In [ ]:
import pandas as pd
searched_results.to_csv(directory + 'searched_results_ontoloy.csv', index=False)

### PyKoSpacing
- 검색이 되지 않은 키워드들에 대해 띄어쓰기를 제대로 하여 다시 검색

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-_b6r3jlf
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-_b6r3jlf
     |████████████████████████████████| 511.8 MB 27 kB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268637 sha256=4c743974530b4ac638ddf254cc27623cf03f6108fcd50632f04153e616dc0f7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-dxwpxe2z/wheels/79/a0/33/16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2


In [ ]:
import pandas as pd

# searched_results = pd.read_csv(directory + 'searched_results_ontoloy.csv')
# searched_results = pd.read_csv(directory + 'searched_results_ontoloy_updated.csv')
searched_results = pd.read_csv(directory + 'searched_results_ontoloy_updated_new_1208.csv')

In [ ]:
import pickle

with open(directory + 'error_list_final.pickle', 'rb') as handle:
    error_list = pickle.load(handle)
with open(directory + 'searched_dict_final.pickle', 'rb') as handle:
    searched_dict = pickle.load(handle)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
from tqdm import tqdm
from pykospacing import Spacing
import pickle

spacing = Spacing()

for i, query_search in tqdm(enumerate(error_list['no_results'])):
    try:
      new_query_search = spacing(query_search).replace(' ', '_')
      if new_query_search != query_search:
          results = sparql_query(new_query_search)
          results_df = pd.json_normalize(results['results']['bindings'])
          searched_results = searched_results.append(results_df.loc[:, results_df.columns.str.contains("value")])
          if results_df.shape[0]>0:
              print(query_search, "=>", new_query_search)
              searched_dict[query_search] = set(results_df['category.value'].apply(lambda x: x.replace('http://ko.dbpedia.org/resource/분류:', '')))
              error_list['no_results'].remove(query_search)
              if query_search in error_list['error']:
                  error_list['error'].remove(query_search)

      if i%30 == 0:
          searched_results.to_csv(directory + 'searched_results_ontoloy_updated.csv', index=False)

    except Exception as ex:
        # new_query_search = spacing(query_search).replace(' ', '_')
        # print(ex)
        print('-----------------> Error:', query_search)

0it [00:00, ?it/s]

1/1 [==============================] - 9s 9s/step


62153it [00:09, 6751.69it/s]

1/1 [==============================] - 0s 21ms/step
물결무늬 => 물결_무늬
1/1 [==============================] - 0s 22ms/step


62193it [00:19, 2574.06it/s]

...

62273it [00:39, 762.20it/s] 

1/1 [==============================] - 0s 21ms/step


62274it [00:40, 745.66it/s]

1/1 [==============================] - 0s 23ms/step
오프라윈프리 => 오프라_윈프리
1/1 [==============================] - 0s 23ms/step


62326it [00:54, 1144.97it/s]


In [ ]:
import pandas as pd
searched_results.to_csv(directory + 'searched_results_ontoloy_updated.csv', index=False)

### Others

- 일부 검색되지 않은 남은 키워드들에 대해 다시 검색 시도

In [ ]:
import pandas as pd

searched_results = pd.read_csv(directory + 'searched_results_ontoloy.csv')
searched_results1 = pd.read_csv(directory + 'searched_results_ontoloy_updated.csv')
searched_results2 = pd.read_csv(directory + 'searched_results_ontoloy_updated_new.csv')
searched_results3 = pd.read_csv(directory + 'searched_results_ontoloy_updated_new_1208.csv')
searched_results4 = pd.read_csv(directory + 'searched_results_ontoloy_updated_new_1208_.csv')
searched_results5 = pd.read_csv(directory + 'searched_results_ontoloy_updated_new_1209.csv')
searched_results6 = pd.read_csv(directory + 'searched_results_ontoloy_updated_new_1209_.csv')

In [ ]:
searched_results3

,subject.value,comment.value,category.value,redirects.value
0,http://ko.dbpedia.org/resource/해산물,해산물(海産物)은 바다에서 얻은 수산 동물들을 두루 일컫는 말이다. 해물로 줄여 부...,http://ko.dbpedia.org/resource/분류:해산물,NaN
1,http://ko.dbpedia.org/resource/해산물,해산물(海産物)은 바다에서 얻은 수산 동물들을 두루 일컫는 말이다. 해물로 줄여 부...,http://ko.dbpedia.org/resource/분류:해산물,NaN
2,NaN,해산물(海産物)은 바다에서 얻은 수산 동물들을 두루 일컫는 말이다. 해물로 줄여 부...,http://ko.dbpedia.org/resource/분류:해산물,http://ko.dbpedia.org/resource/해산물
3,NaN,해산물(海産物)은 바다에서 얻은 수산 동물들을 두루 일컫는 말이다. 해물로 줄여 부...,http://ko.dbpedia.org/resource/분류:해산물,http://ko.dbpedia.org/resource/해산물
4,http://ko.dbpedia.org/resource/국민연금공단,"국민연금공단(國民年金公團, National Pension Service)은 국민연금...",http://ko.dbpedia.org/resource/분류:1987년_설립,NaN
...,...,...,...,...
10528,NaN,"인연(因緣, 산스크리트어: hetu-pratyaya 또는 nidāna)은 원인을 의...",http://ko.dbpedia.org/resource/분류:불교_용어,http://ko.dbpedia.org/resource/인연
10529,NaN,"호국 불교 사상(護國佛敎思想) 또는 호국 사상(護國思想)은 진호국가(鎭護國家), 즉...",http://ko.dbpedia.org/resource/분류:불교_사상,http://ko.dbpedia.org/resource/호국
10530,NaN,"호국 불교 사상(護國佛敎思想) 또는 호국 사상(護國思想)은 진호국가(鎭護國家), 즉...",http://ko.dbpedia.org/resource/분류:한국의_불교,http://ko.dbpedia.org/resource/호국
10531,NaN,"호국 불교 사상(護國佛敎思想) 또는 호국 사상(護國思想)은 진호국가(鎭護國家), 즉...",http://ko.dbpedia.org/resource/분류:불교_사상,http://ko.dbpedia.org/resource/호국


In [ ]:
# searched_results1['subject'] = searched_results1[['subject', 'subject.value', 'redirects.value']].fillna('').sum(axis=1).apply(lambda x: x.replace('http://ko.dbpedia.org/resource/', ''))
searched_results5['subject'] = searched_results5[['subject.value', 'redirects.value']].fillna('').sum(axis=1).apply(lambda x: x.replace('http://ko.dbpedia.org/resource/', ''))
searched_results5 = searched_results5[['subject', 'comment.value', 'category.value']].drop_duplicates()
searched_results5['category.value'] = searched_results5['category.value'].apply(lambda x: x.replace('http://ko.dbpedia.org/resource/분류:', ''))
searched_results5 = searched_results5.groupby(['subject', 'comment.value']).agg({'category.value':lambda x: list(x)}).reset_index()

In [ ]:
searched_results = searched_results.append(searched_results5)

In [ ]:
searched_results = searched_results[['subject', 'comment.value', 'category.value']].drop_duplicates(['subject']).reset_index(drop=True)

In [ ]:
searched_results

,subject,comment.value,category.value
0,11번가,"11번가(11st)는 2008년 2월 사업을 시작한 대한민국 오픈 마켓으로, 201...","['SK그룹', '온라인_서점']"
1,11월,"11월은 그레고리력에서 한 해의 열한 번째 달이며, 30일까지 있다. 이 달과 그 ...",['11월']
2,12월,"12월은 그레고리력에서 한 해의 열두 번째 달이며, 31일까지 있다. 한국, 중국,...",['12월']
3,1차세계대전,"제1차 세계 대전(第一次世界大戰, World War I, 1914년 7월 28일 ~...","['뉴질랜드의_전쟁', '미국의_역사_(1865-1918)', '오스트레일리아의_전..."
4,21세기,21세기는 2001년부터 2100년까지를 말한다.,['21세기']
...,...,...,...
20539,후암동,후암동(厚岩洞)은 서울특별시 용산구에 있는 행정동 및 법정동이다.,"[1946년_설치, 서울특별시의_동, 서울_용산구의_행정_구역]"
20540,후회,"후회(後悔)는 이전에 자신이 내린 결정이 잘못된 것이라고 느끼는 감정이다. 보통, ...",[감정]
20541,휴대폰,"휴대 전화(携帶電話, 문화어: 손전화, 무선대화기) 또는 휴대폰은 이동통신 서비스의...",[휴대_전화기]
20542,휴먼,사람 또는 호모 사피엔스(라틴어: Homo sapiens)는 두 발로 서서 걸어 다...,"[사람, 유인원]"


In [ ]:
searched_results.to_csv(directory + 'searched_results_ontoloy_semifinal.csv', index=False)

In [ ]:
import pandas as pd

searched_results = pd.read_csv(directory + 'searched_results_ontoloy_updated_new_1209.csv')

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
from tqdm import tqdm
import pandas as pd
import pickle

# queries_tosearch = [item[0] for item in Counter([keyword for keywords in metadata['keyword_list'] for keyword in keywords]).most_common(50)]
queries_tosearch = set([keyword for keywords in metadata['keyword_list'] for keyword in keywords]) - set(searched_results['subject'])
# queries_tosearch = set([keyword for keywords in metadata['keyword_list'][0].values() for keyword in keywords]) - set(searched_results['subject'])

print('Search for...', queries_tosearch)

Search for... {'리코타치즈', '자유이용권', '수습사원', '김민희', '협상인사이트', '카레만드는법', '교육컨설팅', '인디에어', '이용', '비즈니스문서', '닭육수', '압축', '기계공학', '담낭', '투자노하우', '팔로우', '정치권력', '중성화수술', '얼굴각질관리방법', '욕실변화', '클렌징오일', '라스트홀리데이', '수리문학상', '하나비', '일코', '츄라우미', '억새물결', '킥오프미팅', '자출', '필요역량', '야간고등학교', '외향적', '네모', 'mbc스페셜', '분노조절', '역사체험', '배경자료', '더치', '애플고딕', '손저림', '세포검사', 'p2p대출법안', '협력사업', '장소', '감량체중유지', '내셔널리그', '방문기', '건축공사', '경력개발경로', '행복의조건', '온천', '동네카페', '기획직무', '정기과정', '영상촬영', '체력측정', '낙엽장수', '공항경찰', '글로벌고수익', '산업혁신', '휘파람', '원격', '스튜디오고민', '사운드플렉스', '스미스', '농부시장', '프로강사', '수록', '수술후기', 'MC', '브런치X빨강머리앤', '발라드', '달걀볶음밥', '사기', '쌍둥이남매', '이완', '해시태그캠페인', '학습능력', '주거환경', '소자본창업', '관광경쟁력', '신기주', '의료기관', '자아강도', '후배', 'ai스피커', '휴직', '퇴사당일', '보약', '보험계약', '그린마케팅', '돈까스클럽', '짐싸기', '노화방지', '핀란드교육현장', '스케쥴', '청춘기록', '인터넷서점', '아라비카', '대마도여행', '한림제지', '농산물', 'cj홈쇼핑', '블랑', '아기', '마약전쟁', '공간디자인', '제품후기', '회계교육', '연애일기', '식단관리', '이분법', '원가', '비영리', '혁신창업', '최남단', '식습관', '예고', '서울진로직업박람회', '연애고민', '재현', '초콜릿책방', '주식

In [ ]:
len(queries_tosearch)

23239

In [ ]:
filtered_list = (set([''.join(sorted(set(q)|set('_'))) for q in queries_tosearch]) - set([''.join(sorted(set(s))) for s in set(searched_results['subject'])])) | set([''.join(sorted(set(q))) for q in queries_tosearch]) - set([''.join(sorted(set(s))) for s in set(searched_results['subject'])])

In [ ]:
queries_tosearch = [query for query in queries_tosearch if ''.join(sorted(set(query))) in filtered_list]

In [ ]:
len(queries_tosearch)

22880

In [ ]:
'백수' in queries_tosearch

True

In [ ]:
sparql_query('에세이')

{'head': {'link': [], 'vars': ['subject', 'comment', 'category', 'redirects']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'comment': {'type': 'literal',
     'xml:lang': 'ko',
     'value': '수필(隨筆) 또는 에세이(essay)는 생각을 자유롭게 표현한 산문 문학이다. 주제에 따라 일상 생활처럼 가벼운 주제를 다루는 경수필과 사회적 문제 등의 무거운 주제를 다루는 중수필로 나뉜다. 특히 중수필에서 사회적 이슈를 주제로 쓴 것을 칼럼이라 한다.'},
    'category': {'type': 'uri',
     'value': 'http://ko.dbpedia.org/resource/분류:수필'},
    'redirects': {'type': 'uri',
     'value': 'http://ko.dbpedia.org/resource/에세이'}}]}}

In [ ]:
import time
searched_results = pd.DataFrame()
searched_dict = {}
no_results = []
# error_list = {'no_results': [], 'error': []}

for i, query_search in tqdm(enumerate(queries_tosearch)):
    try:
        time.sleep(0.3)
        results = sparql_query(query_search)
        results_df = pd.json_normalize(results['results']['bindings'])
        searched_results = searched_results.append(results_df.loc[:, results_df.columns.str.contains("value")])
        if results_df.shape[0]>0:
            searched_dict[query_search] = set(results_df['category.value'].apply(lambda x: x.replace('http://ko.dbpedia.org/resource/분류:', '')))
        else:
            no_results.append(query_search)
        if i%500 == 0: 
            searched_results.to_csv(directory + 'searched_results_ontoloy_updated_new_1209.csv', index=False)
    except Exception as ex: 
        continue

if any(searched_results.columns.str.contains('redirects.value')):
    searched_results = searched_results.drop_duplicates(['redirects.value', 'category.value']).reset_index(drop=True)
if any(searched_results.columns.str.contains('subject.value')):
    searched_results = searched_results.drop_duplicates(['subject.value', 'category.value']).reset_index(drop=True)

23372it [4:37:56,  1.40it/s]


In [ ]:
searched_results.to_csv(directory + 'searched_results_ontoloy_updated_new_1209.csv', index=False)

In [ ]:
import pandas as pd

searched_results1 = pd.read_csv(directory + 'searched_results_ontoloy_semifinal.csv')
# searched_results1['subject'] = searched_results1[['subject', 'subject.value', 'redirects.value']].fillna('').sum(axis=1).apply(lambda x: x.replace('http://ko.dbpedia.org/resource/', ''))
searched_results['subject'] = searched_results[['subject.value', 'redirects.value']].fillna('').sum(axis=1).apply(lambda x: x.replace('http://ko.dbpedia.org/resource/', ''))
searched_results = searched_results[['subject', 'comment.value', 'category.value']].drop_duplicates()
searched_results['category.value'] = searched_results['category.value'].apply(lambda x: x.replace('http://ko.dbpedia.org/resource/분류:', ''))
searched_results = searched_results.groupby(['subject', 'comment.value']).agg({'category.value':lambda x: list(x)}).reset_index()
searched_results1 = searched_results1.append(searched_results)
searched_results1 = searched_results1[['subject', 'comment.value', 'category.value']].drop_duplicates(['subject']).reset_index(drop=True)
searched_results.to_csv(directory + 'searched_results_ontoloy_semifinal_1209_final.csv', index=False)
# searched_results1.to_csv(directory + 'searched_results_ontoloy_semifinal_1209.csv', index=False)

In [ ]:
'에세이' in no_results

False

In [ ]:
sparql_query('고흐')

{'head': {'link': [], 'vars': ['subject', 'comment', 'category', 'redirects']},
 'results': {'distinct': False, 'ordered': True, 'bindings': []}}

In [ ]:
len(no_results)

21818

In [ ]:
import pickle
with open(directory + 'no_results_list_1209.pkl', 'wb') as f:
    pickle.dump(no_results, f)

## Semantic Representation

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 5.8 MB 32.8 MB/s 
     |████████████████████████████████| 1.3 MB 65.2 MB/s 
     |████████████████████████████████| 182 kB 69.2 MB/s 
     |████████████████████████████████| 7.6 MB 49.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=407ea9a00e1823a874b49e3a1a6d9227627f2245ba81021ec84b47f3c387d4ed
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util

'''
https://github.com/snunlp/KR-SBERT
'''

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/394 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

searched_results = pd.read_csv(directory + 'searched_results_ontoloy_semifinal_1209_final.csv')

In [ ]:
searched_results[searched_results['subject'].apply(lambda x: x == '에세이')]

,subject,comment.value,category.value
22678,에세이,수필(隨筆) 또는 에세이(essay)는 생각을 자유롭게 표현한 산문 문학이다. 주제...,['수필']


In [ ]:
len(searched_results['subject'])

23029

In [ ]:
import pickle

sentences = searched_results['comment.value']
vectors = model.encode(sentences) # encode sentences into vectors

with open(directory + 'ontology_comments_vectors_1209.pickle', 'wb') as handle:
    pickle.dump(vectors, handle, protocol=pickle.HIGHEST_PROTOCOL) 

## Document to Embedding vectors
- 자기가 속한 키워드의 comment를 encoding해서 그걸 키워드 벡터로 사용하고, 키워드가 여러개라면 aggregate/pooling (max, min, avg)
* 현재는 mean pooling을 진행한 상황

In [4]:
import pandas as pd

searched_results = pd.read_csv(directory + 'searched_results_ontoloy_semifinal_1209_final.csv')

In [ ]:
import ast
searched_results['category.value'] = searched_results['category.value'].apply(lambda x: ast.literal_eval(x))

In [ ]:
import pickle

with open(directory + 'ontology_comments_vectors_1209.pickle', 'rb') as handle:
    vectors = pickle.load(handle)

In [ ]:
vectors.shape

(23029, 768)

In [ ]:
tmp = dict(zip(searched_results['subject'], vectors))

In [ ]:
import numpy as np

doc_emb_dict = {}
for i, keywords in zip(metadata['id'], metadata['keyword_list']):
  doc_emb_dict[i] = np.nanmean([tmp[k] for k in keywords if k in tmp.keys()])

<ipython-input-9-59c3cf1f7851>:5: RuntimeWarning: Mean of empty slice
  doc_emb_dict[i] = np.nanmean([tmp[k] for k in keywords if k in tmp.keys()])


In [ ]:
metadata[metadata['id'] == '@funemployment_24']

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
149459,40773,@funemployment,퇴사를 묻는 사람들에게,"[퇴사, 에세이, 백수]",https://brunch.co.kr/@funemployment/24,,1543200057000,24,@funemployment_24


In [ ]:
doc_emb_dict['@funemployment_24']

-0.00202981

In [ ]:
empty_keyword_id = metadata.loc[metadata['keyword_list'].apply(lambda x: len(x) == 0), 'id']

In [ ]:
import numpy as np

# any(np.isnan(val) for val in doc_emb_dict.values()) # True
sum(np.isnan(val) for val in doc_emb_dict.values()) / len(doc_emb_dict.keys()) # 0.11757517479015166

0.11757517479015166

In [ ]:
len([key for key, val in doc_emb_dict.items() if np.isnan(val)])

17607

### Case 1. 키워드가 존재하지만 nan 값을 가지는 경우

In [ ]:
ids = [key for key, val in doc_emb_dict.items() if np.isnan(val) and (key not in empty_keyword_id.tolist())]

In [ ]:
len(ids)

9349

In [ ]:
metadata[metadata['id'].apply(lambda x: x in ids[:20])]

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
51,0,@yamestyle,"웹기획에 적합한 사람, 누구?","[웹기획가이드, 웹기획, 야메군]",https://brunch.co.kr/@yamestyle/1,웹기획가이드 - 기획자가 갖춰야 할 마인드는 무엇일까?,1532595692000,1,@yamestyle_1
84,18697,@lloo1135,방랑자,"[방랑자, 안녕, 순간]",https://brunch.co.kr/@lloo1135/50,별 하나로 길을 찾는 일,1474046448000,50,@lloo1135_50
94,0,@urlifestory,법정 스님 - 살아있는 것은 다 행복하라,"[법정, 군생활]",https://brunch.co.kr/@urlifestory/29,,1455774831000,29,@urlifestory_29
186,0,@mrs-ssong,27.Apr.18,"[매일그림, 아이, 성향]",https://brunch.co.kr/@mrs-ssong/18,매일그림- 거울 보기.,1524865099000,18,@mrs-ssong_18
188,29288,@hallym,"똑똑한 선택을 유도하는 부드러운 힘, ‘넛지Nudge’","[선택, 자기계발]",https://brunch.co.kr/@hallym/146,리처드 H. 탈러 <넛지>,1536196350000,146,@hallym_146
197,0,@kimssibang,'좋은 것 같아요'라는 말,"[그림에세이, 공감에세이, 말]",https://brunch.co.kr/@kimssibang/10,주머니탐구생활#07. 꼼지락꼼지락,1546439611000,10,@kimssibang_10
219,0,@bong,일본 학생들의 엄청난 장풍?,"[드래곤볼z, 에너지]",https://brunch.co.kr/@bong/878,,1523951517000,878,@bong_878
291,10888,@intel007,[대충영어] 쉽게 배우는 방법을 제시하다,"[영어공부, 대충영어, 영어교육]",https://brunch.co.kr/@intel007/76,전직 외교관이 전하는 영어공부법,1487405360000,76,@intel007_76
296,3887,@skyscanner,인생샷 건지러 떠나는 오키나와 자유여행,"[오키나와, 오키나와여행, 일본여행]",https://brunch.co.kr/@skyscanner/474,오키나와 인생샷 스팟 5,1547715888000,474,@skyscanner_474
315,3887,@skyscanner,아이슬란드 오로라 여행 준비 A to Z,"[오로라여행, 오로라여행지, 아이슬란드여행]",https://brunch.co.kr/@skyscanner/479,오로라 명소부터 촬영 장비까지,1548318037000,479,@skyscanner_479


#### Co-occurence based Keyword Clustering
- 있는 정보만 활용

In [ ]:
from collections import Counter
from itertools import combinations

counted = Counter([tuple(sorted(k)) for pair in metadata['keyword_list'].apply(lambda x: list(combinations(x, 2))) for k in pair])

In [ ]:
import pandas as pd

counted = pd.DataFrame.from_dict(counted.items())
counted[['n1', 'n2']] = pd.DataFrame(counted.iloc[:,0].tolist(), index=counted.index)
counted.rename(columns={1: 'weight'}, inplace=True)

In [ ]:
counted

,0,weight,n1,n2
0,"(메일, 출판사)",1,메일,출판사
1,"(과거, 생각)",4,과거,생각
2,"(무시, 생각)",3,무시,생각
3,"(과거, 무시)",1,과거,무시
4,"(세계여행, 여행)",217,세계여행,여행
...,...,...,...,...
206975,"(멕시코, 파추카)",1,멕시코,파추카
206976,"(식물, 책임감)",1,식물,책임감
206977,"(책임감, 화분)",1,책임감,화분
206978,"(경험수집, 독서모임)",1,경험수집,독서모임


In [ ]:
keywords_list = metadata.loc[metadata['id'].apply(lambda x: x in ids), ['id', 'keyword_list']]

In [ ]:
keywords_list.head(3)

,id,keyword_list
51,@yamestyle_1,"[웹기획가이드, 웹기획, 야메군]"
84,@lloo1135_50,"[방랑자, 안녕, 순간]"
94,@urlifestory_29,"[법정, 군생활]"


In [ ]:
co_occ_lst = [keyword for keywords in keywords_list['keyword_list'].tolist() for keyword in keywords]
co_occ_lst = counted[counted.apply(lambda x: (x['n1'] in co_occ_lst) | (x['n2'] in co_occ_lst), axis=1)]

In [ ]:
co_occ_lst.head(3)

,0,weight,n1,n2
0,"(메일, 출판사)",1,메일,출판사
4,"(세계여행, 여행)",217,세계여행,여행
5,"(세계여행, 행복)",6,세계여행,행복


In [ ]:
def find_cooccurence(co_occ_lst_, klst, top_k=5):
    co_occ_lst_ = co_occ_lst_[co_occ_lst_.apply(lambda x: (x['n1'] in klst) | (x['n2'] in klst), axis=1)]
    co_occ_lst_['target'] = co_occ_lst_.apply(lambda x: 'Both' if ((x['n1'] in klst) & (x['n2'] in klst)) else \
                                                    x['n1'] if (x['n1'] in klst) else \
                                                    x['n2'] if (x['n2'] in klst) else None, axis=1)
    res = co_occ_lst_[co_occ_lst_['target'].apply(lambda x: (x != 'Both') & (x != None))].sort_values(by='weight', ascending=False)
    if res.shape[0] > 0:
        res = res.groupby('target').apply(lambda x: set([k for keys in x.head(top_k)[0] for k in keys]))
        return set([k for key in res.reset_index()[0] for k in key])
    else:
        return []

In [ ]:
import pickle
from tqdm import tqdm

for i, klst in tqdm(zip(keywords_list['id'], keywords_list['keyword_list'])):
    try:
        if np.isnan(doc_emb_dict[i]):
            # co_occ_lst_ = co_occ_lst.copy()
            co_occ_lst_ = counted.copy()
            keywords = find_cooccurence(co_occ_lst_, klst, top_k=5)
            doc_emb_dict[i] = np.nanmean([tmp[k] for k in keywords if k in tmp.keys()])
    except:
        with open(directory + 'document_embedding_ontology.pkl', 'wb') as f:
            pickle.dump(doc_emb_dict, f)

0it [00:00, ?it/s]<ipython-input-19-56464feb5ee2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  co_occ_lst_['target'] = co_occ_lst_.apply(lambda x: 'Both' if ((x['n1'] in klst) & (x['n2'] in klst)) else \
<ipython-input-21-31dce60160d0>:10: RuntimeWarning: Mean of empty slice
  doc_emb_dict[i] = np.nanmean([tmp[k] for k in keywords if k in tmp.keys()])
9349it [13:48, 11.29it/s]


In [ ]:
import pickle

with open(directory + 'document_embedding_ontology.pkl', 'wb') as f:
    pickle.dump(doc_emb_dict, f)

#### Keyword - Category Matrix (사용 X)
- 키워드의 '분류' 정보를 매트릭스로 나타냄

In [ ]:
from collections import Counter

Counter([s for ss in searched_results['category.value'] for s in ss]).most_common(10)

[('살아있는_사람', 248),
 ('한국_요리', 93),
 ('법률용어', 76),
 ('민법', 62),
 ('대한민국의_텔레비전_배우', 59),
 ('대한민국의_영화_배우', 57),
 ('감정', 53),
 ('영어계_외래어', 49),
 ('한국어_영화_작품', 48),
 ('음식', 48)]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
doc = searched_results['category.value'].apply(lambda x: ' '.join(x))
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(doc)

In [ ]:
print(X.toarray().shape)

(23029, 25022)


In [ ]:
searched_results.shape

(23029, 3)

In [ ]:
import numpy as np

tmp_dict = {}
s = set([val for vals in searched_dict.values() for val in vals])

for k, v in searched_dict.items():
    tmp_dict[k] = [1 if s_ in v else 0 for s_ in s]

KeyboardInterrupt: ignored

In [ ]:
TDM = pd.DataFrame.from_dict(tmp_dict,orient='index')
TDM.columns = s

In [ ]:
TDM

In [ ]:
from scipy.spatial.distance import cdist

np.where(((1 - cdist(TDM, TDM, metric='cosine')) - np.eye(TDM.shape[0])) == 1)

(array([ 0, 52]), array([52,  0]))

In [ ]:
TDM.index[0], TDM.index[52]

('여행', '배낭여행')

### Case 2. 키워드가 존재하지 않는 경우

In [5]:
import pickle

with open(directory + 'document_embedding_ontology.pkl', 'rb') as f:
    doc_emb_dict = pickle.load(f)

- 해당 특허 외에서 함께 등장하는 키워드들이 없는 경우와
- 아예 키워드가 없는 경우는 다음과 같음

In [ ]:
import numpy as np
sum([np.isnan(v) for v in doc_emb_dict.values()]) # 8566
len([key for key, val in doc_emb_dict.items() if np.isnan(val) and (key in empty_keyword_id.tolist())]) # 8258

8566

In [ ]:
set([key for key, val in doc_emb_dict.items() if np.isnan(val) and (key in empty_keyword_id.tolist())]) - set([key for key, val in doc_emb_dict.items() if np.isnan(val)]) # 포함관계

set()

In [23]:
import numpy as np
target_id = set([key for key, val in doc_emb_dict.items() if np.isnan(val)])

#### Similarities Between Vectors  
- text 간 유사도 산출하여 유사한 글들에서부터 키워드 가져옴
- text에서 키워드 추출하기에는 활용할 수 있는 정보가 매우 부족
- magazine keyword를 가져오는 방안도 고려해보기

In [13]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 5.8 MB 87.6 MB/s 
     |████████████████████████████████| 1.3 MB 89.0 MB/s 
     |████████████████████████████████| 182 kB 99.1 MB/s 
     |████████████████████████████████| 7.6 MB 60.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b36f6b2372f9bc29e59ad5c2e98ac6e6ff8b5dec7031aa8813dfd86f219d28b6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [6]:
metadata['text'] = metadata['title'] + '. ' + metadata['sub_title']

In [7]:
metadata = metadata[['id', 'text', 'keyword_list']]

In [ ]:
metadata.loc[metadata['id'].apply(lambda x: x in target_id), 'text'].head(10)

62                                       개인으로 남은 자의 변명. 
74           사운드 오브 뮤직 속으로. 신랑이랑 유럽여행 다섯째 날.-오스트리아 잘츠부르크
111                                                나은이. 
121                                            아찔한 순간들. 
123        5탄 타래료,청폐안의 원작 작가 정묵, 그녀는 누구?. 랑랑이가 소개하는 문학작품
139                                        what a mess. 
150    전파의 소음 없는 고요한 시간 앞에서.. 정읍 안진사 고택에서 보낸 몇 년 전의 기...
175                                         현실적인 조언 모음. 
183          15화. 자기 주관을 가지고 녹여내다.. 주관이 있는 사람은 경쟁력이 생긴다.
200               어느 아마추어 배우의 첫 연기 연습. 순수하나 정말 바보같은 이유이지
Name: text, dtype: object

In [46]:
focal = metadata.loc[metadata['id'].apply(lambda x: x in target_id), ['id', 'text']]
others = metadata.loc[metadata['id'].apply(lambda x: x not in target_id), ['id', 'text']]
sentences1 = focal['text'].tolist()
sentences2 = others['text'].tolist()

In [14]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 
vectors1 = model.encode(sentences1) # encode sentences into vectors
vectors2 = model.encode(sentences2) # encode sentences into vectors

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/394 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [15]:
similarities = util.cos_sim(vectors1, vectors2) # compute similarity between sentence vectors
# print(similarities)

In [ ]:
similarities.shape

torch.Size([8566, 141185])

In [16]:
import pickle

with open(directory + 'similarities_btw_text.pkl', 'wb') as f:
    pickle.dump(similarities, f)

In [17]:
del sentences1
del sentences2
del vectors1
del vectors2
del model
del doc_emb_dict

import gc
gc.collect()

0

In [9]:
import pickle

with open(directory + 'similarities_btw_text.pkl', 'rb') as f:
    similarities = pickle.load(f)

In [16]:
import numpy as np
ind = np.argsort(-similarities)[:, :3] # Top 3개 유사한 게시글의 emb 값 추출

In [18]:
ind.shape

torch.Size([8566, 3])

In [28]:
len(sentences1)

8566

In [30]:
len(sentences2)

141185

In [58]:
from tqdm import tqdm

tmp = []
for i, id in tqdm(enumerate(focal['id'])):
  for j in ind[i]:
    tmp.append(doc_emb_dict[others.reset_index(drop=True).id[j.item()]])
  doc_emb_dict[id] = np.nanmean(tmp)

In [59]:
set([key for key, val in doc_emb_dict.items() if np.isnan(val)])

set()

In [60]:
import pickle

with open(directory + 'document_embedding_ontology_final.pkl', 'wb') as f:
    pickle.dump(doc_emb_dict, f)